In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
import numpy as np

In [2]:
# Data
source_texts = ['hello', 'how are you', 'goodbye']
target_texts = ['bonjour', 'comment ça va', 'au revoir']

In [3]:
# Tokenize and pad sequences
source_tokenizer = Tokenizer()
target_tokenizer = Tokenizer()
source_tokenizer.fit_on_texts(source_texts)
target_tokenizer.fit_on_texts(target_texts)

In [4]:
# Text to sequences
source_sequences = source_tokenizer.texts_to_sequences(source_texts)

source_texts,source_sequences

(['hello', 'how are you', 'goodbye'], [[1], [2, 3, 4], [5]])

In [5]:
# pad sequences
source_sequences = pad_sequences(source_sequences, padding='post')

source_sequences

array([[1, 0, 0],
       [2, 3, 4],
       [5, 0, 0]])

In [6]:
target_sequences = target_tokenizer.texts_to_sequences(target_texts)
target_sequences = pad_sequences(target_sequences, padding='post')

target_sequences

array([[1, 0, 0],
       [2, 3, 4],
       [5, 6, 0]])

In [7]:
# Reshape target sequences for sparse_categorical_crossentropy
target_sequences = np.expand_dims(target_sequences, -1)

target_sequences

array([[[1],
        [0],
        [0]],

       [[2],
        [3],
        [4]],

       [[5],
        [6],
        [0]]])

In [8]:
X= source_sequences
X.reshape(X.shape[0], X.shape[1], 1)
rnn_shape = (X.shape[1],1)

In [9]:
# Model
model = Sequential([
    SimpleRNN(16, return_sequences=True, input_shape=rnn_shape),
    Dense(len(target_tokenizer.word_index) + 1, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

C:\Users\ahmed\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
# Training
model.fit(X, target_sequences, epochs=100)

Epoch 1/100


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 2.0431
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 2.0297
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 2.0166
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 2.0036
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 1.9908
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 1.9781
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 1.9656
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 1.9534
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 1.9412
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 1.9293
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 1.9175
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 1.9058
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 1.8943
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 1.8829
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 1.8716
Epoch 16/100
1/1 ━━━━━━━━━━━━━━━━

In [11]:
# Inference function
def translate(text):
    sequence = source_tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=source_sequences.shape[1], padding='post')
    prediction = model.predict(padded_sequence)

    translated_sentence = []
    for word_probs in prediction[0]:
        predicted_word_index = np.argmax(word_probs)
        word = target_tokenizer.index_word.get(predicted_word_index, "")
        if word:
            translated_sentence.append(word)
    return ' '.join(translated_sentence)

In [12]:
# Test translation
print(translate("hello"))
print(translate("how are you"))
print(translate("goodbye"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
au ça va
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
au revoir


In [13]:
target_tokenizer.index_word.setdefault


<function dict.setdefault(key, default=None, /)>